<center>
<h1>Welcome to the Lab 🥼🧪</h1>
</center>

### How can I analyze supply, demand, and price trends for new construction?

In this notebook, we will be creating an analysis on new construction. We will be comparing new construction demand relative to the price of new construction sales for the US housing market. This notebook will work for any location in the US, whether it be a zip code, city, or metro area. To fine tune it for your location, modify the search criteria. 

#### What will you create in this notebook?

<p align="center">
  <img src="../../../images/new_construction_pricing_and_demand.png" alt="Alt text">
</p>

#### Need help getting started?

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along.

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY`.

Run in collab --> [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-cookbook/blob/main/examples/housing_market_research/supply_and_demand/new_construction_trends.ipynb)

### 1. Import the Parcl Labs Python Library

In [ ]:
# if needed, install and/or upgrade to the latest verison of the Parcl Labs Python library
%pip install --upgrade parcllabs==1.10.0 nbformat

In [2]:
import os
import pandas as pd
from parcllabs import ParclLabsClient
from parcllabs.beta.charting.styling import SIZE_CONFIG
from parcllabs.beta.charting.default_charts import create_dual_axis_chart

client = ParclLabsClient(
    api_key=os.environ.get('PARCL_LABS_API_KEY', "<your Parcl Labs API key if not set as environment variable>"), 
    limit=12 # set default limit
)

### 2. Search for Markets

In [ ]:
# in this case, lets look at US market overall
us_market = client.search.markets.retrieve(
    query='United States',
    sort_by='TOTAL_POPULATION',
    sort_order='DESC',
    limit=1
)

us_market.head()

In [4]:
us_parcl_id = us_market['parcl_id'].values[0]

### 3. Retrieve the Data

In [ ]:
# lets get new construction demand and supply counts in addition to prices
new_construction_housing_event_prices = client.new_construction_metrics.housing_event_prices.retrieve(
    parcl_ids=[us_parcl_id],
    limit=100, # lets get the full series
)

new_construction_housing_event_counts = client.new_construction_metrics.housing_event_counts.retrieve(
    parcl_ids=[us_parcl_id],
    limit=100
)

In [ ]:
new_construction_housing_event_prices.head()

In [ ]:
new_construction_housing_event_counts.head()

### 4. Prepare the data for analysis/charting

In [ ]:
# in this notebook, we will focus on median sales price and demand counts
# lets merge the two dataframes
new_construction = pd.merge(new_construction_housing_event_prices[['parcl_id', 'date', 'price_median_sales']], new_construction_housing_event_counts[['parcl_id', 'date', 'sales', 'new_listings_for_sale']], on=['date', 'parcl_id'], how='inner')
new_construction

### 5. Chart the Data

In [ ]:
new_construction = new_construction.sort_values('date')
# rename sales to Number of Units Sold for readability
new_construction = new_construction.rename(columns={'sales': 'Number of Units Sold'})
new_construction.head()

In [ ]:
create_dual_axis_chart(
    title='US New Construction Median Sales Price vs. Number of Units Sold',
    line_data=new_construction,
    bar1_data=new_construction,
    bar2_data=new_construction,
    line_series='price_median_sales',
    bar1_series='Number of Units Sold',
    yaxis1_title='Median Sales Price',
    yaxis2_title='Number of Units Sold',
    height=SIZE_CONFIG['x']['height'],
    width=SIZE_CONFIG['x']['width']
)